In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

We want to get data from the Wikipedia "Lists of American Films" pages. Each of these pages has a URL following the same genral pattern, and generally consists of a few similarly structured tables holding at the very least the name of a movie. 

There are a couple inconsistancies between the different years' pages:
1. Movies before 1900 have a single page and are held in a single table with multi-column spans grouping the different years.
2. Starting around 2014, there is are multi-row spans for the film's opening month: there is a tacky vertical box spelling out the month name vertically over multiple rows. Worse still the day of the month is then in a seperate column with multi-row spans, and these two columns are grouped together into a single "Opening" column. 
3. Other pages have tables split up by first letter of the name of the film, and don't necessarily have the same column names from table to table, even within a single year's page.

In [85]:
html = """
<table>
<tr><th colspan="2">Opening<th>
    <th style="width:20%;">Title</th>
    <th style="width:10%;">Director</th>
    <th>Cast</th>
    <th style="width:13%">Genre</th>
    <th style="width:20%">Notes</th>
    <th>Ref.</th>
</tr>

<tr>
    <th rowspan="22"><b>J<br />A<br />N<br />U<br />A<br />R<br />Y<br /></b></th>
    <th><b>2</b></th>
    <td><i><a href="/wiki/The_Woman_in_Black_2:_Angel_of_Death" class="mw-redirect" title="The Woman in Black 2: Angel of Death">The Woman in Black 2: Angel of Death</a></i></td>
    <td><a href="/wiki/Tom_Harper_(director)" title="Tom Harper (director)">Tom Harper</a></td>
    <td><a href="/wiki/Phoebe_Fox" title="Phoebe Fox">Phoebe Fox</a><p><a href="/wiki/Jeremy_Irvine" title="Jeremy Irvine">Jeremy Irvine</a></p><p><a href="/wiki/Helen_McCrory" title="Helen McCrory">Helen McCrory</a></p><p><a href="/wiki/Adrian_Rawlins" title="Adrian Rawlins">Adrian Rawlins</a></p><p><a href="/wiki/Ned_Dennehy" title="Ned Dennehy">Ned Dennehy</a></p></td>
    <td><a href="/wiki/Horror_film" title="Horror film">Horror</a></td>
    <td><a href="/wiki/Relativity_Media" title="Relativity Media">Relativity Media</a><p>Sequel to <i><a href="/wiki/The_Woman_in_Black_(2012_film)" title="The Woman in Black (2012 film)">The Woman in Black (2012)</a></i></p></td>
    <td></td>
</tr>

<tr>
    <th rowspan="2"><b>9</b></th> 
    <td><i><a href="/wiki/Taken_3" title="Taken 3">Taken 3</a></i></td>
    <td><a href="/wiki/Olivier_Megaton" title="Olivier Megaton">Olivier Megaton</a></td>
    <td><a href="/wiki/Liam_Neeson" title="Liam Neeson">Liam Neeson</a><p><a href="/wiki/Forest_Whitaker" title="Forest Whitaker">Forest Whitaker</a></p><p><a href="/wiki/Famke_Janssen" title="Famke Janssen">Famke Janssen</a></p><p><a href="/wiki/Maggie_Grace" title="Maggie Grace">Maggie Grace</a></p></td>
    <td><a href="/wiki/Action_film" title="Action film">Action</a></td>
    <td><a href="/wiki/20th_Century_Fox" title="20th Century Fox">20th Century Fox</a></td>
    <td></td>
</tr>
</table>
"""

In [96]:
soup = BeautifulSoup(html, 'html.parser')
# Get a table to work with
tables = soup.find_all('table') #, {'class': 'wikitable'})
table = tables[0]

# Get all rows from table, i.e., all <tr> tags
rows = table.find_all('tr')

# row of headers
row0 = rows[0]
# row with stupid month column
row1 = rows[1]
# row with stupid day column
row2 = rows[2]
# normalish row
# row3 = rows[3]

# Check if row is a header row and if so, build the columns.
# Should be refactored to be more pythonic?
# I Want list comprehensions...
row = row0

if not row.find_all('td'):
    # If no <td> tags then a header row
    columns = []
    for x in row.find_all('th'):
        colspan = int(x.attrs.pop('colspan', 1))
        if colspan > 1:
            columns += [f'{x.text.strip()}__{i}' for i in range(colspan)]
        else:
            columns += [x.text.strip()]

def parse_header_row(row):
    """If a row is a header, run this."""
    columns = []
    for x in row.find_all('th'):
        colspan = int(x.attrs.pop('colspan', 1))
        if colspan > 1:
            columns += [f'{x.text.strip()}__{i}' for i in range(colspan)]
        else:
            columns += [x.text.strip()]
    return columns
            
            
# For entries with rowspan, save value and amount left to fill
counters = dict((key, [0, None]) for key in columns)

def parse_data_row(row, columns, counters):
    """If a row is not a header, run this."""
    cells = row.find_all(['th', 'td'])
    cell_cursor = 0
    row_processed = []

    for col in columns:

        print('cursor', cell_cursor, '\ncolumn', col)
        print(counters)

        # Check if values to propagate
        if counters[col][0] > 0:
            print('in the counter if')
            
            cell_value = counters[col][1]

            counters[col][0] -= 1   
        # If not propagate, get from cell    
        else:
            cell = cells[cell_cursor]
            rowspan = int(cell.attrs.pop('rowspan', 1))
            cell_value = cell.text.strip()
            print('rowspan', rowspan, '\ncell_value', cell_value)

            if rowspan > 1:
                counters[col] = [rowspan - 1, cell_value]
                
            cell_cursor += 1

        row_processed.append(cell_value)     
        
    return row_processed
    

### FOR SOME REASON THE ATTRIBUTES ARTE GETTING LOST ON MULTIPLE RUNS?!


# row0.find_all(['th', 'td'])
# [cell.attrs for cell in row1.find_all('th')]

# if all cells have <th colspan="n"> w/ n \ge 0 then header row
# instantiate column for each and multiple for those with colspan > 0

# if at least one <th rowspan> or <td> (table data) tag, then data row
# if rowspan > 0 then instantiate counter and set that column equal to that value for the 
#     next rowspan rows. Multiple counters should be handled ?appropriately?


In [140]:
def _parse_header_row(row):
    """If a row is a header, run this."""
    
    # If exists <td> tags then not a header row
    if row.find_all('td'):
        raise ValueError("`row` is not a table header.")
    
    columns = []
    for x in row.find_all('th'):
        colspan = int(x.attrs.pop('colspan', 1))
        if colspan > 1:
            columns += [f'{x.text.strip()}__{i}' for i in range(colspan)]
        else:
            columns += [x.text.strip()]
    return columns
            

def _parse_data_row(row, columns, counters):
    """If a row is not a header, run this."""
    cells = row.find_all(['th', 'td'])
    cell_cursor = 0
    row_processed = []

    for col in columns:
        # Check if values to propagate
        if counters[col][0] > 0:
            cell_value = counters[col][1]
            counters[col][0] -= 1   
        # If not propagate, get from cell    
        else:
            cell = cells[cell_cursor]
            rowspan = int(cell.attrs.pop('rowspan', 1))
            cell_value = cell.text.strip()

            if rowspan > 1:
                counters[col] = [rowspan - 1, cell_value]
                
            cell_cursor += 1

        row_processed.append(cell_value)     
        
    return row_processed


def parse_table(table):
    """Parse rows of table."""
    rows = table.find_all('tr')

    header_row = rows.pop(0)
    columns = _parse_header_row(header_row) 
    # For entries with rowspan, save value and amount left to fill        
    counters = dict((key, [0, None]) for key in columns)
    
    table_parsed = []
    for row in rows:
        row_parsed = _parse_data_row(row, columns, counters)
        table_parsed.append(row_parsed)
        
    table_parsed = pd.DataFrame(table_parsed, columns=columns)
    return table_parsed

In [141]:
url = f'https://en.wikipedia.org/wiki/List_of_American_films_of_2015'
# Get raw html
response = requests.get(url)  
# Soupify
soup = BeautifulSoup(response.text, 'html.parser')

# Get a table to work with
tables = soup.find_all('table', {'class': 'wikitable'})
table = tables[0]

parse_table(table).head()

,Opening__0,Opening__1,Title,Director,Cast,Genre,Notes,Ref.
0,JANUARY,2,The Woman in Black 2: Angel of Death,Tom Harper,Phoebe Fox\nJeremy Irvine\nHelen McCrory\nAdri...,Horror,Relativity Media\nSequel to The Woman in Black...,
1,JANUARY,9,Taken 3,Olivier Megaton,Liam Neeson\nForest Whitaker\nFamke Janssen\nM...,Action,20th Century Fox,
2,JANUARY,9,Let's Kill Ward's Wife,Scott Foley,Scott FoleyPatrick WilsonDonald FaisonJames Ca...,Comedy,Well Go USA Entertainment,[11]
3,JANUARY,14,Match,Stephen Belber,Patrick StewartCarla GuginoMatthew LillardRob ...,Drama,IFC Films,[12]
4,JANUARY,16,Blackhat,Michael Mann,Chris HemsworthViola DavisManny MontanaTang Wei,Action,Universal Pictures,[13]


Okay, now we just need a giant list of either movie names or IMDB ids...

# EVERYTHING BELOW IS OLD AND OUTDATED

### 1.2. Getting a List of Movies

Using `requests` we now have a way of extracting data about a film, given its title. Now all we need is a list of movies over which to iterate. After a bit of googling, I found a wikipedia page listing movies, but it is organized through several subpages. We'll use the BeautifulSoup package to crawl these pages to generate our list of movies.

There are two paths of attack:
1. `https://en.wikipedia.org/wiki/Lists_of_films` has data on every movie ever made. There are a few different hierarchies movies are classified by on this. This catalogs every movie ever made.
2. `https://en.wikipedia.org/wiki/Lists_of_American_films` is just American films, organized by year. This is maybe easier to work with, since the organization is more orderly.

Since this project is ultimately stupid, I'll just do the easier thing and use the American movie list. Additionally, the American film list is split up by year, where the different years have their own pages. The urls for these pages are very consistent, so easy to crawl. Each year's page has the movies listed in tables, which appear to always have the same columns.

For this part I used this tutorial https://goo.gl/Bm1cdD (sort of).

A webpage is basically a tree with nodes labeled by tags. BeautifulSoup packages this tree in an object that is easy to navigate and search, based off of the html tags attached to the nodes. Here's how we are going to pull the movie data for any given year.
1. We pull the raw html using requests and passing the url to a `GET` request.
2. Dump the html string into a BeautifulSoup object.
3. From inspecting the raw html it looks like the data we want is always in a table, which we can find with the `table` tag. There are some other tables, such as the footers and page navigation that we don't want. Again from inspection, it looks like the tables we do want always have `class=wikitable`.
4. The tables are then split into rows. A row either consitsts of headers (`th` table header tag) or data (`td` table data tag). We pull this, put it in a data frame if we actually get data, and skip if we get an error putting it into the data frame.

When we first wrote the `fetch_movie_data` function below, we did not have the `drop_colspan` feature Or the 1919 correction. When it was ranon all time, there were a few problems:
1. Starting in 2014, the movie lists have an "Opening" data column, whose entries span multiple rows. Even though our script didn't throw any errors for 2015 the data gathered is wrong: actors are listed as movie titles, titles are listed as opening dates, and so on. These cells have tags with colspan attributes, which are larger than 1. To fix this we added the `drop_colspan` flag, which does not include columns that span multiple cells.
2. There is a "typo" on the page for 1919. The very last row of the table has an extra cell. Not sure what the best way to deal with this one edge case is. I ended up counting the number of columns read, and only take that many cells for the data rows.

There are definitely better ways to handle these problems. In particular, my solution assumes the multicolumns are all up front in the table, and that the multicolumn and the 1919 typo don't coincide on any page.

In [71]:
def fetch_movie_data(url, drop_colspan=True):
    # Get the response from GET request
    response = requests.get(url)  
    # Throw error if API call has an error
    if response.status_code != 200:
        raise requests.HTTPError(
            f'Couldn\'t call API. Error {response.status_code}.'
        )
  
    soup = BeautifulSoup(response.text, 'html.parser')

    # Get all tables on the site where the class is wikitable
    # This prevents it from including layout tabes, like the wikipedia footer etc
    tables = soup.find_all('table', {'class': 'wikitable'})

    fetched = []
    for table in tables:
        # Get all table rows (tr) tag
        rows = table.find_all('tr')
        # Assuming first row is headers, look for table header (th) tags
        if drop_colspan:
            columns = []; drop = 0
            for col in rows[0].find_all('th'):
                # If has an attribute for colspan > 1, don't include
                try:
                    int(col.attrs['colspan']) > 1
                    drop += int(col.attrs['colspan'])
                except:
                    columns.append(col.text.strip())
        else:
            columns = [x.text.strip() for x in rows[0].find_all('th')]
        
        # Assuming remaining rows are data, look for table data (td) tags
        data = [[x.text.strip() for x in row.find_all('td')] for row in rows[1:]]
        
        if drop_colspan:
            # Assuming the multicols are all in front
            if drop > 0:
                data = [row[-len(columns):] for row in data]
        # Deal with error in the 1919 Wikipedia page...
        # data = [row[:len(columns)] for row in data]
        # Make sure we got data. 
        # If for whatever reason there is an error, don't include
        if data and columns:
            try:
                df = pd.DataFrame(data, columns=columns)
            except:
                continue
            fetched.append(df)
    return pd.concat(fetched, sort=False)

In [72]:
movies_1994 = fetch_movie_data('https://en.wikipedia.org/wiki/List_of_American_films_of_1952')

In [74]:
movies_1994.tail()

,Title,Director,Cast,Genre,Notes/Studio,Notes
91,Yankee Buccaneer,Frederick de Cordova,"Jeff Chandler, Scott Brady, Suzan Ball",Adventure,NaN,Universal
92,You for Me,Don Weis,"Jane Greer, Peter Lawford",Romance,NaN,MGM
93,Young Man with Ideas,Mitchell Leisen,"Glenn Ford, Ruth Roman",Comedy,NaN,MGM
94,Yukon Gold,Frank McDonald,"Kirby Grant, Martha Hyer",Western,NaN,Monogram
95,Zombies of the Stratosphere,Fred C. Brannon,,Serial,NaN,Republic


Now that we've gotten things to appear to work for one year, let's creep that net. The above tutorial suggests using a timer to prevent us from sending too many requests to Wikipedia and getting blocked. I'm expecting the `fetch_movie_data` to fail sometimes, so let's collect our data in a dict. That way we can know exactly what years failed and what years we have data for.

In [ ]:
# For each year from 1900 to 2017
movie_data = {}
for year in range(1900, 2018):
    url = f'https://en.wikipedia.org/wiki/List_of_American_films_of_{year}'
    try:
        data = fetch_movie_data(url)
        movie_data[year] = data
    except:
        print(f'Something went wrong fetching data for {year}')
        pass
    # Pause for 2 sec to not overwhelm Wikipedia
    time.sleep(2)

This took maybe 10 minutes to run. 

We really only want the title and the year, so let's get that. We could also get the cast and genre from here, but I feel like the OMDb versions of this will be more consistent.

In [ ]:
for year, movies in movie_data.items():
    movies['Year'] = year
movies_full = pd.concat([movies[['Title', 'Year']] for movies in movie_data.values()])

In [ ]:
len(movies_full)

We have ~26K movies, which would take about 26 days to get IMDb data for using the free OMDb API. Maybe we should kick them a buck to do this faster.

### 1.3. Combining Everything

Note to future self: When running this we got fewer results when requesting full synopsis. The API only return results with a full synposis if requested.

In [ ]:
all_teh_jsons = []
for row in movies_full.itertuples(index=False, name=None):
    try:
        response_json = get_movie_data(*row)
    except:
        # If can't find with year, don't use
        try:
            response_json = get_movie_data(row[0])
        except:
            # Continue if no data found
            continue
        if response_json['Plot'] != 'N/A':
            all_teh_jsons.append(response_json)